In [ ]:
import os
import boto3

# Obtener el nombre del endpoint de SageMaker desde variables de entorno
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']
runtime = boto3.client('runtime.sagemaker')

# Mapeo de clases (índice a etiqueta)
CLASS_MAPPING = {0: 'low_value', 1: 'medium_value', 2: 'high_value'}

def lambda_handler(event, context):
    try:

        payload = event.get('predictions', '').replace(" ", "")  # Eliminar espacios adicionales


        if not payload:
            raise ValueError("La clave 'predictions' no tiene valores válidos.")

        # Invocar el endpoint de SageMaker
        response = runtime.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType='text/csv',
            Body=payload
        )

        # Leer el índice de la clase predicha
        predicted_class_index = int(float(response['Body'].read().decode('utf-8').strip()))


        return CLASS_MAPPING.get(predicted_class_index, "unknown")

    except Exception as e:
        print(f"Error: {str(e)}")
        return "error"
